In [ ]:
import pandas as pd
import seaborn as sns
import os
import analysis

In [ ]:
analyser = analysis.Analyser("k6")
all_data = analyser.get_data()
all_data

In [ ]:
req_duration_data = all_data[
    all_data["metric_name"].isin(["http_req_duration", "grpc_req_duration"])
]
req_duration_data.columns

In [ ]:
# drop setup data
req_duration_data = req_duration_data[req_duration_data["group"] != "::setup"]

In [ ]:
# filter to only http2 data
# req_duration_data = req_duration_data[req_duration_data["http_version"] == 2]

In [ ]:
data = req_duration_data.copy()
parts = data["name"].str.split("?", expand=True).rename(columns={0: "path", 1: "query"})
parts["path"] = parts["path"].str.lstrip("https://127.0.0.1:8000")
req_duration_data["path"] = parts["path"]
if "query" in parts:
    req_duration_data["query"] = parts["query"]
else:
    req_duration_data["query"] = [None for _ in range(len(parts["path"]))]

In [ ]:
# normalise endpoints from path
req_duration_data["endpoint"] = req_duration_data["path"].map(
    lambda x: x.split("/")[-1].lower().replace("_", "")
)

In [ ]:
ignore_vars = [
    "metric_name",
    "metric_value",
    "service",
    "subproto",
    "proto",
    "scenario",
    "status",
    "tls_version",
    "url",
    "name",
    "extra_tags",
    "metadata",
    "check",
    "error",
    "error_code",
    "expected_response",
    "group",
    "method",
    "query",
    "path",
    "node_ips",
]

In [ ]:
data = req_duration_data.copy()
if len(data["query"].dropna()) > 0:
    var, invariant_vars = analysis.condense_vars(data, ignore_vars + ["path"])
    data["vars"] = var

    txids = data["query"].str.split("=", expand=True)[1]
    data["txids"] = txids
    txid_counts = data.groupby(["txids", "vars"]).size()
    txid_counts = pd.DataFrame(txid_counts)
    txid_counts.rename(columns={0: "committed_count"}, inplace=True)

    ax = sns.displot(kind="ecdf", data=txid_counts, x="committed_count", hue="vars")
    ax.set(title="number of commit checks before committed")
    filename = f"commit_checks_ecdf-committed_count-None-None-vars"
    ax.savefig(os.path.join(analyser.plot_dir(), f"{filename}.jpg"))

In [ ]:
data = req_duration_data.copy()
if len(data["query"].dropna()) > 0:
    var, invariant_vars = analysis.condense_vars(data, ignore_vars + ["path"])
    data["vars"] = var

    grouped = data.groupby(["path", "query", "vars"])

    starts = grouped.min("start_ms")
    ends = grouped.max("end_ms")

    commit_latency_ms = ends["end_ms"] - starts["start_ms"]

    latencies = pd.DataFrame(commit_latency_ms)
    latencies.rename(columns={0: "commit_latency"}, inplace=True)

    ax = sns.displot(kind="ecdf", data=latencies, x="commit_latency", hue="vars")
    ax.set(title="commit latency")
    filename = f"commit_latency_ecdf-commit_latency-None-None-vars"
    ax.savefig(os.path.join(analyser.plot_dir(), f"{filename}.jpg"))

In [ ]:
# req_duration_data = req_duration_data[req_duration_data["rate"] == 100]
# req_duration_data = req_duration_data[req_duration_data["vus"] == 10]
# req_duration_data = req_duration_data[req_duration_data["func"] == "mixed_single"]
# req_duration_data.describe()

In [ ]:
# skip the start and avoid high latency connection setups
req_duration_data = req_duration_data[req_duration_data["start_ms"] > 1000]

In [ ]:
plot_data = req_duration_data.copy(deep=False)
plot_data = plot_data[plot_data["content_type"] == "json"]
plot_data = plot_data[plot_data["rate"] <= 200]
plot_data = plot_data[plot_data["enclave"] == "virtual"]
p = analyser.plot_scatter(plot_data, col="http_version", ignore_vars=ignore_vars)
p.figure.suptitle("")
p.set(xlabel="start time (ms)", ylabel="latency (ms)")

In [ ]:
plot_data = req_duration_data.copy(deep=False)
plot_data = plot_data[plot_data["content_type"] == "json"]
plot_data = plot_data[plot_data["rate"] <= 200]
p = analyser.plot_ecdf(plot_data, col="endpoint", ignore_vars=ignore_vars)
p.figure.suptitle("")
p.set(xlabel="latency (ms)")

In [ ]:
plot_data = req_duration_data.copy(deep=False)
p = analyser.plot_percentile_latency_over_time(
    data, col="http_version", ignore_vars=ignore_vars, percentile=0.99
)
p.figure.suptitle("")
p.set(xlabel="time (ms)", ylabel="achieved throughput (req/s)")

In [ ]:
plot_data = req_duration_data.copy(deep=False)
p = analyser.plot_throughput_over_time(
    plot_data, col="http_version", ignore_vars=ignore_vars, interval=1000
)
p.figure.suptitle("")
p.set(xlabel="time (ms)", ylabel="achieved throughput (req/s)")

In [ ]:
plot_data = req_duration_data.copy(deep=False)
plot_data = plot_data[plot_data["path"] != "app/tx"]
# plot_data = plot_data[plot_data["http_version"] ==2]
analyser.plot_achieved_throughput_bar(
    plot_data, col="content_type", ignore_vars=ignore_vars
)

In [ ]:
plot_data = req_duration_data.copy(deep=False)
plot_data = plot_data[plot_data["http_version"] == 1]
plot_data = plot_data[plot_data["nodes"] == 1]
p = analyser.plot_throughput_bar(plot_data, ignore_vars=ignore_vars + ["endpoint"])
p.figure.suptitle("")
p.set(xlabel="target throughput (req/s)", ylabel="achieved throughput ratio")

In [ ]:
plot_data = req_duration_data.copy(deep=False)
# plot_data = plot_data[plot_data["http_version"] == 1]
p = analyser.plot_throughput_bar(plot_data, ignore_vars=ignore_vars + ["endpoint"])
p.figure.suptitle("")
p.set(xlabel="target throughput (req/s)", ylabel="achieved throughput ratio")

In [ ]:
plot_data = req_duration_data.copy(deep=False)
plot_data = plot_data[plot_data["path"] != "app/tx"]
plot_data = plot_data[plot_data["http_version"] == 1]
p = analyser.plot_target_throughput_latency_line(
    plot_data, ignore_vars=ignore_vars + ["endpoint"]
)
p.figure.suptitle("")